In [1]:
# @title 1. Cài đặt thư viện
!pip install -q -U torch transformers peft datasets bitsandbytes trl accelerate
!pip install -q openpyxl pandas scikit-learn
print("Đã cài đặt xong!")

Đã cài đặt xong!


In [2]:
## 🚀 Sửa đổi: Chỉ Upload File CSV & Xử lý Data Train
import pandas as pd
from google.colab import files
from datasets import Dataset
import io

# 1. Upload
print("📂 Chọn file CSV (.csv) chứa **Dữ liệu Huấn luyện (Train Data)** của bạn:")
uploaded = files.upload()
filename = next(iter(uploaded))

# 2. Đọc file CSV & Lấy cột 2, 3
try:
    # Đọc file CSV. Bạn có thể cần điều chỉnh 'encoding' và 'sep' (dấu phân cách) nếu file của bạn không phải là UTF-8 và dấu phẩy (,)
    # Ví dụ: nếu dùng dấu chấm phẩy: pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')), header=None, sep=';')
    df = pd.read_csv(io.BytesIO(uploaded[filename]), header=None, encoding='utf-8')

    # Bỏ dòng tiêu đề nếu cần (giả sử dòng 0 là tiêu đề thì bỏ comment dòng dưới)
    # df = df.iloc[1:]

    # Lấy cột 2 (câu hỏi, index 1) và cột 3 (câu trả lời, index 2)
    df = df.iloc[:, [1, 2]]
    df.columns = ['question', 'answer']
    df = df.dropna()

    # Gán cho train_df
    train_df = df

    print(f"✅ Đã tải và xử lý xong {len(train_df)} dòng dữ liệu huấn luyện.")

except Exception as e:
    print(f"❌ Lỗi khi đọc file CSV: {e}")
    # Hiển thị 5 dòng đầu của file (nếu đọc được) để kiểm tra encoding/separator
    try:
        sample_content = uploaded[filename].decode('utf-8').split('\n')[:5]
        print("\n📝 5 dòng đầu của file để kiểm tra cấu trúc (separator/encoding):")
        for line in sample_content:
             print(line)
    except:
        pass
    # raise e # uncomment this line if you want to stop execution on error


# 3. Format chuẩn Qwen cho tập Train
def format_qwen(row):
    """
    Định dạng một cặp Hỏi-Đáp theo chuẩn ChatML của Qwen.
    <|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n{answer}<|im_end|>
    """
    # Đảm bảo là string và loại bỏ khoảng trắng dư thừa
    q = str(row['question']).strip()
    a = str(row['answer']).strip()
    return f"<|im_start|>user\n{q}<|im_end|>\n<|im_start|>assistant\n{a}<|im_end|>"

# Tạo Dataset train
if 'train_df' in locals() and not train_df.empty:
    train_dataset = Dataset.from_pandas(train_df)
    train_dataset = train_dataset.map(lambda x: {"text": format_qwen(x)})

    # Xóa cột '__index_level_0__' do pandas to_dataset tạo ra
    train_dataset = train_dataset.remove_columns([col for col in train_dataset.column_names if col.startswith('__index_level_0__')])

    print("✅ Data Train (format Qwen) đã sẵn sàng! Có thể truy cập bằng biến: `train_dataset`")
    print("\n---")
    print("✨ Dòng dữ liệu mẫu sau khi format:")
    print(train_dataset[0]['text'])
else:
    print("❌ Không thể tạo `train_dataset`. Dữ liệu trống hoặc lỗi đọc file.")

📂 Chọn file CSV (.csv) chứa **Dữ liệu Huấn luyện (Train Data)** của bạn:


Saving train.csv to train (3).csv
✅ Đã tải và xử lý xong 233 dòng dữ liệu huấn luyện.


Map:   0%|          | 0/233 [00:00<?, ? examples/s]

✅ Data Train (format Qwen) đã sẵn sàng! Có thể truy cập bằng biến: `train_dataset`

---
✨ Dòng dữ liệu mẫu sau khi format:
<|im_start|>user
question<|im_end|>
<|im_start|>assistant
answer<|im_end|>


In [3]:
# @title 3. Load Model & Config (Safe Mode Float32)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

# --- THAY ĐỔI QUAN TRỌNG NHẤT ---
# Dùng float32 cho tính toán trung gian.
# Nó tốn VRAM hơn xíu nhưng né được hoàn toàn lỗi BFloat16 trên T4.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,  # <--- Dùng Float32 thay vì Float16
    bnb_4bit_use_double_quant=True,
)

print("⏳ Đang tải model (Safe Mode)...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Load model với torch_dtype=float16 (để lưu trữ), nhưng tính toán sẽ là float32 (do config trên)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

# Config LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

⏳ Đang tải model (Safe Mode)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

trainable params: 29,933,568 || all params: 3,115,872,256 || trainable%: 0.9607


In [4]:
# @title 4. Start Training - Fixed for T4
from trl import SFTTrainer, SFTConfig
import os

OUTPUT_DIR = "Qwen2.5-3B-Finetuned"

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    dataset_text_field="text",
    max_length=512,  # Đổi từ max_length thành max_seq_length
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,

    # ✅ CRITICAL FIX: Tắt gradient scaling để tránh bfloat16 error
    fp16=False,  # Tắt FP16 scaling
    bf16=False,  # Tắt BF16

    logging_steps=5,
    logging_strategy="steps",
    save_strategy="no",
    report_to="none",
    optim="paged_adamw_32bit",

    # Thêm để tránh NaN gradients
    max_grad_norm=1.0,
    gradient_checkpointing=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    processing_class=tokenizer,  # Dùng processing_class thay vì tokenizer
    peft_config=peft_config,
)

print("🚀 Bắt đầu train...")
trainer.train()

print("💾 Đang lưu model...")
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Đã xong! Model lưu tại: {OUTPUT_DIR}")

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/233 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/233 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/233 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


🚀 Bắt đầu train...


Step,Training Loss
5,1.883800
10,1.561500
15,1.610200
20,1.387800
25,1.405200
30,1.438200


💾 Đang lưu model...
✅ Đã xong! Model lưu tại: Qwen2.5-3B-Finetuned


In [5]:
## 🚀 Sửa đổi: Nén và Tải Model đã Finetune về máy
import shutil
from google.colab import files
import os

# Tên thư mục chứa model sau khi finetune (giả sử)
MODEL_DIR = "Qwen2.5-3B-Finetuned"
# Tên file zip kết quả
zip_name = "Finetuned_Model_Package"

print(f"📦 Đang nén thư mục model: {MODEL_DIR}...")

# 1. Kiểm tra sự tồn tại của model
if not os.path.exists(MODEL_DIR):
    print(f"❌ Lỗi: Không tìm thấy thư mục model '{MODEL_DIR}'.")
    print("Vui lòng kiểm tra lại tên thư mục model sau khi finetune.")
else:
    # 2. Nén thư mục model thành zip
    # shutil.make_archive(base_name, format, root_dir)
    # root_dir là thư mục gốc mà các file sẽ được nén từ đó
    # base_name là tên file zip (không bao gồm đuôi .zip)
    try:
        shutil.make_archive(zip_name, 'zip', '.', MODEL_DIR)

        # 3. Tải xuống
        final_zip_file = f"{zip_name}.zip"
        print(f"✅ Đã nén thành công: {final_zip_file}")
        print(f"⬇️ Đang tải xuống {final_zip_file} ...")
        files.download(final_zip_file)

    except Exception as e:
        print(f"❌ Lỗi trong quá trình nén/tải xuống: {e}")

📦 Đang nén thư mục model: Qwen2.5-3B-Finetuned...
✅ Đã nén thành công: Finetuned_Model_Package.zip
⬇️ Đang tải xuống Finetuned_Model_Package.zip ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
!pip install peft -U

In [11]:
# 2. Tải model và tokenizer (ĐÃ SỬA LẠI CÁCH TẢI VÀ OFFLOAD)
print("⏳ Đang tải Model và Tokenizer (đã cấu hình offload)...")
try:
    # 1. Thiết lập device_map: Sử dụng 'auto' là đúng.
    # 2. Xử lý offload: Chỉ định rõ ràng offload lên CPU.
    # 3. Độ chính xác: Vẫn dùng torch.float16

    tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL_PATH)

    # Sử dụng cách tải model tối ưu cho bộ nhớ hạn chế
    model = AutoModelForCausalLM.from_pretrained(
        FINETUNED_MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        # Thêm tham số offload nếu cần thiết (để accelerate quản lý)
        # Nếu model vẫn lớn, bạn có thể buộc nó offload lên CPU
        # offload_state_dict = True,
        # offload_buffers = True,
    )
    model.eval()

    # 3. Tạo Pipeline cho Chat
    # Vẫn KHÔNG DÙNG tham số 'device' trong pipeline!
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    print("✅ Model và Pipeline đã sẵn sàng.")

except Exception as e:
    print(f"❌ Lỗi khi tải model hoặc tokenizer: {e}")
    pipe = None

⏳ Đang tải Model và Tokenizer (đã cấu hình offload)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Model và Pipeline đã sẵn sàng.
